In [2]:
from data_getters import *

In [3]:
# Primero se construye el dataset con el cuál se va a trabajar
data = get_macro_dataframe()

### Obtención de datos y variables agregadas
La data tiene las siguientes variables de base:
- Inflación Chile
- Inflación USA
- TPM Chile
- TPM USA
- IPC Chile
- IPC USA
- S&P 500
- IPSA
- USDCLP


Se agregan los siguientes indicadores:

- Categoría TPM (Baja, Media, Alta)
- Dummies de cambio en la TPM (1 si cambió ese día)
- Dummies de nuevos máximos en las Bolsas (1 si hay nuevo max en SP500 o IPSA)
- Diferencial de Tasas de Política Monetaria.
- Dummies de si tasa pertenece a una categoría.
- Dummy de si el diferencial es positivo o no.

In [4]:
data.columns

Index(['Fecha', 'inflacion_cl', 'inflacion_us', 'tpm_cl', 'tpm_us', 'ipc_cl',
       'ipc_us', 'sp500', 'ipsa', 'usdclp', 'tpm_cl_cat', 'tpm_us_cat',
       'tpm_cl_is_0', 'tpm_us_is_0', 'tpm_cl_is_1', 'tpm_us_is_1',
       'tpm_cl_is_2', 'tpm_us_is_2', 'tpm_cl_change', 'tpm_us_change',
       'new_max_sp500', 'new_max_ipsa', 'new_max_usdclp', 'tpm_diff',
       'tpm_diff_pos'],
      dtype='object')

### Variable a predecir
Se define el direccional como :

$$
y_t = USDCLP_{t+1}-USDCLP_{t} > 0
$$

Por lo que se busca aplicar un modelo de clasificación que permita predecir si el closing del USDCLP del día siguiente va a ser más alto que el de hoy.

In [5]:
# Definimos nuestra variable objetivo
isup = (data['usdclp'].shift(-1) - data['usdclp']) > 0

# Lo agregamos al dataframe
data['isup'] = isup

# Eliminamos filas con valores nulos (en este caso el último día)
data = data.dropna()

#### Split y Entrenamiento

In [6]:
data.columns

Index(['Fecha', 'inflacion_cl', 'inflacion_us', 'tpm_cl', 'tpm_us', 'ipc_cl',
       'ipc_us', 'sp500', 'ipsa', 'usdclp', 'tpm_cl_cat', 'tpm_us_cat',
       'tpm_cl_is_0', 'tpm_us_is_0', 'tpm_cl_is_1', 'tpm_us_is_1',
       'tpm_cl_is_2', 'tpm_us_is_2', 'tpm_cl_change', 'tpm_us_change',
       'new_max_sp500', 'new_max_ipsa', 'new_max_usdclp', 'tpm_diff',
       'tpm_diff_pos', 'isup'],
      dtype='object')

In [7]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Definimos las features y la variable objetivo
features = [c for c in data.columns if c not in ['isup', 'usdclp', "Fecha"]]
X = data[features]
y = data['isup']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [9]:
# Hacemos el scaling de los features separando X_train y X_test, no queremos data leakage
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# Se entrenan los modelos
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

model_ab = AdaBoostClassifier(n_estimators=100, random_state=42)
model_ab.fit(X_train, y_train)
y_pred_ab = model_ab.predict(X_test)

model_mlp = MLPClassifier(hidden_layer_sizes=(50, 25), max_iter=500, random_state=42)
model_mlp.fit(X_train, y_train)
y_pred_mlp = model_mlp.predict(X_test)

In [11]:
# Metrics 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def print_metrics(y_test, y_pred, model_name):
    print(f"Metrics for {model_name}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
    print(f"ROC AUC: {roc_auc_score(y_test, y_pred):.4f}")
    print("\n")

In [12]:
print_metrics(y_test, y_pred_rf, "Random Forest")
print_metrics(y_test, y_pred_ab, "AdaBoost")
print_metrics(y_test, y_pred_mlp, "MLP Classifier")

Metrics for Random Forest:
Accuracy: 0.5330
Precision: 0.3333
Recall: 0.0095
F1 Score: 0.0185
ROC AUC: 0.4966


Metrics for AdaBoost:
Accuracy: 0.5683
Precision: 0.5946
Recall: 0.2095
F1 Score: 0.3099
ROC AUC: 0.5433


Metrics for MLP Classifier:
Accuracy: 0.5022
Precision: 0.4726
Recall: 0.6571
F1 Score: 0.5498
ROC AUC: 0.5130




In [13]:
# Se guarda el MLP Classifier ya que es el que mejor resultado tuvo en términos de F1-Score
import joblib
joblib.dump(model_mlp, 'model.pkl')

# Se guarda el scaler
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']